# Extrator de PDF - Implantação

Extração de texto, figuras, e renderização em arquivos PDF. Utiliza a biblioteca [PyMuPDF](https://pymupdf.readthedocs.io/en/latest/intro.html), e consegue extrair partes ou seções selecionadas do texto e de páginas.

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [1]:
%%writefile Model.py
import joblib
import nltk
import pandas as pd
import numpy as np
import base64
import cv2
import logging
import io
from pdf_extractor import PDFExtractor, read_memory, read_file

class Model:
    def __init__(self):
        self.loaded = False;

    def load(self):
        artifacts = joblib.load("/tmp/data/pdf_extractor.joblib")
        self.extractor_parameters = artifacts["extractor_parameters"]
        
        if self.extractor_parameters["initial_page"] != None: 
            self.extractor_parameters["initial_page"] -= 1;

        if self.extractor_parameters["final_page"] != None: 
            self.extractor_parameters["final_page"] -= 1;
        
        self.loaded = True
        print("Loaded model")

    def predict(self, X, feature_names, meta=None):
        if not self.loaded:
            self.load()
        
        if isinstance(X, bytes):
            result = read_memory(X,
                                extract = self.extractor_parameters["extract"],
                                text_filter_begin = self.extractor_parameters["text_filter_begin"],
                                text_filter_end = self.extractor_parameters["text_filter_end"],
                                initial_page = self.extractor_parameters["initial_page"],
                                final_page = self.extractor_parameters["initial_page"])
        else:
            return ""
                                        
        if self.extractor_parameters["extract"] == "text":
            return result
    
        elif self.extractor_parameters["extract"] == "figures":
            result_bytes = []
            if len(result) > 0:
                for img in result:
                    buff = io.BytesIO()
                    img.save(buff, format="JPEG")
                    result_bytes.append(buff.getvalue().decode("latin1"))

                return result_bytes;
            else:
                return result
    
        elif self.extractor_parameters["extract"] == "prints":
            result_bytes = []
            if len(result) > 0:
                for img in result:
                    buff = io.BytesIO()
                    img.save(buff, format="JPEG")
                    result_bytes.append(buff.getvalue().decode("latin1"))

                return result_bytes;
            else:
                return result

Overwriting Model.py
